#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [3]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [4]:
!pip install folium

   ---------------------------------------- 0.0/108.4 kB ? eta -:--:--
   ----------- --------------------------- 30.7/108.4 kB 660.6 kB/s eta 0:00:01
   ----------------------------- --------- 81.9/108.4 kB 919.0 kB/s eta 0:00:01
   -------------------------------------- 108.4/108.4 kB 892.5 kB/s eta 0:00:00


Распакуем наши данные из архива.

"unzip" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


Загрузим данные из csv файла в датафрейм.

In [56]:
df = pd.read_csv("input_data.csv", encoding='utf-8', sep=';')
df

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,56.041539,92.753133,0,0,660030.0,581436.0,24,857003.0
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,55.169949,61.519210,0,0,454079.0,274414.0,74,1820769.0
11358147,2021-12-31,850000,2,2,2,37.0,5.0,55.946206,43.088179,0,0,606101.0,190983.0,52,958329.0
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,61.256383,73.435919,0,0,628406.0,581702.0,86,2156710.0


Отобразим на карте координаты наших построек.

In [57]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [58]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  
    
    # Преобразуем градусы в радианы
    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)
    
    # Разница между координатами
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Формула гаверсинуса
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Возвращаем расстояние в километрах
    return R * c

moscow_center = (55.7558, 37.6176)
spb_center = (59.9343, 30.3351)

# Рассчитываем расстояние до Москвы и Санкт-Петербурга
df['distance_to_moscow'] = haversine_distance(df['geo_lat'], df['geo_lon'], moscow_center[0], moscow_center[1])
df['distance_to_spb'] = haversine_distance(df['geo_lat'], df['geo_lon'], spb_center[0], spb_center[1])

# Создаем признаки is_Moscow и is_Saint_Peterburg
df['is_Moscow'] = df['distance_to_moscow'] <= 20  # True, если расстояние <= 20 км от Москвы
df['is_Saint_Peterburg'] = df['distance_to_spb'] <= 20  # True, если расстояние <= 20 км от Санкт-Петербурга

# Убираем временные столбцы, если они больше не нужны
df.drop(['distance_to_moscow', 'distance_to_spb'], axis=1, inplace=True)

# Посмотрим на результат

print(df[['geo_lat', 'geo_lon', 'is_Moscow', 'is_Saint_Peterburg']])

            geo_lat    geo_lon  is_Moscow  is_Saint_Peterburg
0         56.780112  60.699355      False               False
1         44.608154  40.138381      False               False
2         55.540060  37.725112      False               False
3         44.608154  40.138381      False               False
4         44.738685  37.713668      False               False
...             ...        ...        ...                 ...
11358145  56.041539  92.753133      False               False
11358146  55.169949  61.519210      False               False
11358147  55.946206  43.088179      False               False
11358148  61.256383  73.435919      False               False
11358149  55.164522  61.386448      False               False

[11358150 rows x 4 columns]


In [59]:
df


,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0,False,False
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,False,False
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0,False,False
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,False,False
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,56.041539,92.753133,0,0,660030.0,581436.0,24,857003.0,False,False
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,55.169949,61.519210,0,0,454079.0,274414.0,74,1820769.0,False,False
11358147,2021-12-31,850000,2,2,2,37.0,5.0,55.946206,43.088179,0,0,606101.0,190983.0,52,958329.0,False,False
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,61.256383,73.435919,0,0,628406.0,581702.0,86,2156710.0,False,False


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [60]:
del df['geo_lat']
del df['geo_lon']
del df['object_type']
del df['postal_code']
del df['street_id']
del df['id_region']
del df['house_id']



In [61]:
df

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,False,False
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,False,False
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,False,False
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,False,False
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,False,False
...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.0,0.0,0,False,False
11358146,2021-12-31,2490000,1,10,2,56.9,0.0,0,False,False
11358147,2021-12-31,850000,2,2,2,37.0,5.0,0,False,False
11358148,2021-12-31,4360000,5,5,1,36.0,9.0,0,False,False


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: building_type is_Moscow is_Saint_Peterburg
Числовые: date price level levels rooms area kitchen_area

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [67]:
df_cat = df[['building_type','is_Moscow','is_Saint_Peterburg']]
from sklearn.preprocessing import OneHotEncoder
# Инициализация OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Применяем OneHotEncoder к категориальным столбцам
encoded_columns = encoder.fit_transform(df_cat[['building_type', 'is_Moscow', 'is_Saint_Peterburg']])

# Создаем DataFrame с закодированными признаками
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['building_type', 'is_Moscow', 'is_Saint_Peterburg']))

# Объединяем закодированные признаки с исходным DataFrame
df_encoded = pd.concat([df_cat.drop(columns=['building_type', 'is_Moscow', 'is_Saint_Peterburg']), encoded_df], axis=1)

# Выводим результат
df_encoded

,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,is_Moscow_True,is_Saint_Peterburg_True
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
11358145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11358146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11358147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11358148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [68]:
df['date'] = pd.to_datetime(df['date'])

# 2. Найдем минимальную дату
min_date = df['date'].min()

# 3. Добавим колонку с количеством дней со дня первого наблюдения
df['days_since_first_observation'] = (df['date'] - min_date).dt.days

# 4. Добавим колонку с отношением этажа к количеству этажей
df['floor_to_levels_ratio'] = df['level'] / df['levels']

# 5. Удалим колонку date
df.drop(columns=['date'], inplace=True)

# Посмотрим на результат
print(df.head())

      price  level  levels  rooms  area  kitchen_area  building_type  \
0   2451300     15      31      1  30.3           0.0              0   
1   1450000      5       5      1  33.0           6.0              0   
2  10700000      4      13      3  85.0          12.0              3   
3   3100000      3       5      3  82.0           9.0              0   
4   2500000      2       3      1  30.0           9.0              3   

   is_Moscow  is_Saint_Peterburg  days_since_first_observation  \
0      False               False                             0   
1      False               False                             0   
2      False               False                             0   
3      False               False                             0   
4      False               False                             0   

   floor_to_levels_ratio  
0               0.483871  
1               1.000000  
2               0.307692  
3               0.600000  
4               0.666667  


In [71]:
df_num = df[['price','level','levels','rooms','area','kitchen_area','days_since_first_observation','floor_to_levels_ratio']]
df_num
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numeric_columns = ['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'days_since_first_observation', 'floor_to_levels_ratio']

df_num[numeric_columns] = scaler.fit_transform(df_num[numeric_columns])

print(df_num)

ValueError: Input X contains infinity or a value too large for dtype('float64').

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [ ]:
class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)

    def _predict(self, x):
        # Вычисляем расстояния от x до всех точек в обучающем наборе
        distances = self._compute_distances(x)
        
        # Получаем индексы k ближайших соседей
        k_indices = np.argsort(distances)[:self.n_neighbors]
        
        # Получаем значения y для k ближайших соседей
        k_nearest_values = [self.y_train[i] for i in k_indices]
        
        # Возвращаем среднее значение целевых значений k ближайших соседей
        return np.mean(k_nearest_values)

    def _compute_distances(self, x):
        if self.metric == 'euclidean':
            # Используем евклидово расстояние
            distances = np.linalg.norm(self.X_train - x, axis=1)
        else:
            raise ValueError("Метрика не поддерживается.")
        return distances

**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [ ]:

class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None
        
        self.momentum_weights = None
        self.momentum_bias = None
        self.adagrad_weights = None
        self.adagrad_bias = None
        
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        if self.optimization == 'Momentum':
            self.momentum_weights = np.zeros(n_features)
            self.momentum_bias = 0
        elif self.optimization == 'AdaGrad':
            self.adagrad_weights = np.zeros(n_features)
            self.adagrad_bias = 0
            
        for _ in range(self.max_iter):
            y_predicted = np.dot(X, self.weights) + self.bias
            
            dw = (1/n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1/n_samples) * np.sum(y_predicted - y)

            if self.optimization == 'SGD':
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db
            
            elif self.optimization == 'Momentum':
                self.momentum_weights = self.decay_rate * self.momentum_weights + (1 - self.decay_rate) * dw
                self.momentum_bias = self.decay_rate * self.momentum_bias + (1 - self.decay_rate) * db
                self.weights -= self.learning_rate * self.momentum_weights
                self.bias -= self.learning_rate * self.momentum_bias
            
            elif self.optimization == 'AdaGrad':
                self.adagrad_weights += dw ** 2
                self.adagrad_bias += db ** 2
                adjusted_learning_rate = self.learning_rate / (np.sqrt(self.adagrad_weights) + self.epsilon)
                self.weights -= adjusted_learning_rate * dw
                self.bias -= adjusted_learning_rate * db

    def predict(self, X):
        """Предсказание целевых значений для новых данных X."""
        return np.dot(X, self.weights) + self.bias

    def mean_squared_error(self, y_true, y_pred):
        """Вычисление среднеквадратичной ошибки."""
        return np.mean((y_true - y_pred) ** 2)


# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.